In [141]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

import sys
sys.path.insert(1, '/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo')
from custom_utils import *

In [142]:
df = pd.read_csv("/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo/data/cooked_data/cooked_complete_dataset.csv")
# prevent 0 division zero
df = df.replace(0, 1)

In [143]:
df

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates
0,2020-12-14,19246.64453,19144.49219,19305.09961,19012.70898,2.247400e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12,0.188275,0.297238,2167.931034,134533587.6
1,2020-12-15,19417.07617,19246.91992,19525.00781,19079.84180,2.674198e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18,0.144389,0.399427,2288.857143,133351912.2
2,2020-12-16,21310.59766,19418.81836,21458.90820,19298.31641,4.440901e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11,0.137256,0.489673,2204.314685,132323572.3
3,2020-12-17,22805.16211,21308.35156,23642.66016,21234.67578,7.137861e+10,369.449982,176.740005,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10,0.156723,0.636030,2399.077519,132373208.7
4,2020-12-18,23137.96094,22806.79688,23238.60156,22399.81250,4.038790e+10,367.974792,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2,0.166419,0.107093,2392.031847,131791042.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2021-04-02,59384.31250,59098.87890,60267.18750,58869.28130,5.870000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.121955,10,0.146423,0.152625,2184.200000,163507250.5
110,2021-04-03,57603.89060,59397.41020,60110.26950,57603.89060,5.960000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.501192,1,0.132042,0.274113,2150.511450,165320614.4
111,2021-04-04,58758.55470,57604.83980,58913.74610,57168.67580,5.080000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.206128,1,0.216657,0.148517,1684.979452,162156064.8
112,2021-04-05,59057.87890,58760.87500,59891.29690,57694.82420,6.070000e+10,406.359985,161.919998,1.061571,0.152288,1.176817,1.383126,0.009035,0.179058,21,0.232894,0.351100,1852.858974,161456161.7


In [144]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [145]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD', 'High_BTC-USD',
       'Low_BTC-USD', 'Volume_BTC-USD', 'Adj_Close_SPY', 'Adj_Close_GLD',
       'Adj_Close_CHFUSD=X', 'Adj_Close_CNYUSD=X', 'Adj_Close_EURUSD=X',
       'Adj_Close_GBPUSD=X', 'Adj_Close_JPYUSD=X', 'coindesk_sentiment',
       'num_of_coindesk_posts', 'reddit_comments_sentiments',
       'top_50_reddit_posts_sentiments', 'blockchain_transactions_per_block',
       'blockchain_hash_rates'],
      dtype='object')

In [128]:
# conduct log return transformation on all columns
df = log_return_transformation(df)

In [146]:
# custom number of lags for selected columns
# columns that are not specify in this dictionary will be lagged by on by default
lags = {
    'Adj_Close_BTC-USD': 6,
    'Open_BTC-USD': 6,
    'Low_BTC-USD': 6,
    'Volume_BTC-USD': 6,
    'Adj_Close_SPY': 6,
    'Adj_Close_GLD': 6,
}

In [147]:
df = get_num_lags(df, lags)

In [148]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD_lag_1', 'Open_BTC-USD_lag_2',
       'Open_BTC-USD_lag_3', 'Open_BTC-USD_lag_4', 'Open_BTC-USD_lag_5',
       'High_BTC-USD_lag_1', 'Low_BTC-USD_lag_1', 'Low_BTC-USD_lag_2',
       'Low_BTC-USD_lag_3', 'Low_BTC-USD_lag_4', 'Low_BTC-USD_lag_5',
       'Volume_BTC-USD_lag_1', 'Volume_BTC-USD_lag_2', 'Volume_BTC-USD_lag_3',
       'Volume_BTC-USD_lag_4', 'Volume_BTC-USD_lag_5', 'Adj_Close_SPY_lag_1',
       'Adj_Close_SPY_lag_2', 'Adj_Close_SPY_lag_3', 'Adj_Close_SPY_lag_4',
       'Adj_Close_SPY_lag_5', 'Adj_Close_GLD_lag_1', 'Adj_Close_GLD_lag_2',
       'Adj_Close_GLD_lag_3', 'Adj_Close_GLD_lag_4', 'Adj_Close_GLD_lag_5',
       'Adj_Close_CHFUSD=X_lag_1', 'Adj_Close_CNYUSD=X_lag_1',
       'Adj_Close_EURUSD=X_lag_1', 'Adj_Close_GBPUSD=X_lag_1',
       'Adj_Close_JPYUSD=X_lag_1', 'coindesk_sentiment_lag_1',
       'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1',
       'top_50_reddit_posts_sentiments_lag_1',
       'blo

In [149]:
train = df[df['date']<'2021-03-11'].sort_values('date')
test = df[df['date']>='2021-03-11'].sort_values('date')

In [150]:
train.to_csv('clean_trainingset.csv', index=False)

In [151]:
y_train = train['Adj_Close_BTC-USD']
X_train = train.iloc[:,2:]
X_train_no_sentiment = X_train.drop(['coindesk_sentiment_lag_1', 'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1', 'top_50_reddit_posts_sentiments_lag_1'], axis=1)


y_test = test['Adj_Close_BTC-USD']
X_test = test.iloc[:,2:]
X_test_no_sentiment = X_test.drop(['coindesk_sentiment_lag_1', 'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1', 'top_50_reddit_posts_sentiments_lag_1'], axis=1)


### Linear regression with complete set of independent variables (With sentiments)

In [152]:
model = LinearRegression(normalize=True)
model.fit(X_train, y_train)

LinearRegression(normalize=True)

In [153]:
output_with_sentiment = []
y_pred_return = model.predict(X_train)
output_with_sentiment = output_with_sentiment + list(y_pred_return)
print("RMSE in training set: {}".format(math.sqrt(mean_squared_error(y_pred_return, y_train))))

y_pred_return = model.predict(X_test)
output_with_sentiment = output_with_sentiment + list(y_pred_return)
print("RMSE in training set: {}".format(math.sqrt(mean_squared_error(y_pred_return, y_test))))

RMSE in training set: 1417.124322418379
RMSE in training set: 10035.223394188453


### Linear regression with complete set of independent variables (Without sentiments)

In [154]:
model = LinearRegression(normalize=True)
model.fit(X_train_no_sentiment, y_train)

LinearRegression(normalize=True)

In [155]:
output_without_sentiment = []
y_pred_return = model.predict(X_train_no_sentiment)
output_without_sentiment = output_without_sentiment + list(y_pred_return)
print("RMSE in training set: {}".format(math.sqrt(mean_squared_error(y_pred_return, y_train))))

y_pred_return = model.predict(X_test_no_sentiment)
output_without_sentiment = output_without_sentiment + list(y_pred_return)
print("RMSE in training set: {}".format(math.sqrt(mean_squared_error(y_pred_return, y_test))))

RMSE in training set: 1456.0698350906964
RMSE in training set: 8537.348388121236


In [156]:
predictions = {
    'prediction_with_sentiments': output_with_sentiment,
    'predictions_without_sentiments': output_without_sentiment,
    'date': list(train['date']) + list(test['date'])
}

In [157]:
pd.DataFrame(predictions).to_csv('linear_predictions.csv', index=False)